<a href="https://colab.research.google.com/github/dustinak80/Deep_Learning/blob/master/HW3/HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 3 - Stock price prediction

---



## Import packages

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.initializers import Constant

## Import the Data

In [4]:
stocks = pd.read_csv('https://raw.githubusercontent.com/dustinak80/Deep_Learning/master/HW3/stocks_dataset.csv')
stocks.head()

,Date,ABT,ABBV,ABMD,A,ALXN,ALGN,AGN,ABC,AMGN,ANTM,BAX,BDX,BIIB,BSX,BMY,CAH,CNC,CERN,CI,COO,CVS,DHR,DVA,XRAY,EW,GILD,HCA,HSIC,HOLX,HUM,IDXX,ILMN,INCY,ISRG,IQV,JNJ,LH,LLY,MCK,MDT,MRK,MTD,MYL,PKI,PRGO,PFE,DGX,REGN,RMD,STE
0,2015-01-02 00:00:00,44.900002,65.889999,37.310001,40.560001,186.600006,56.200001,259.130005,90.459999,159.889999,125.459999,39.820751,139.860001,342.010010,13.22,59.509998,80.510002,26.110001,65.029999,102.830002,160.610001,95.099998,64.988625,75.830002,51.930000,63.860001,94.910004,73.970001,53.662746,26.379999,142.990005,73.665001,185.419998,73.760002,175.190002,59.110001,104.519997,111.419998,70.059998,207.199997,71.879997,57.189999,301.950012,56.349998,44.090000,166.630005,31.33,68.970001,410.160004,57.169998,65.160004
1,2015-01-05 00:00:00,44.910000,64.650002,37.070000,39.799999,182.169998,56.990002,256.690002,89.690002,157.990005,123.260002,38.832157,140.720001,336.739990,13.81,58.580002,80.050003,26.040001,64.169998,101.059998,159.369995,94.160004,64.344200,74.699997,51.570000,63.889999,96.790001,71.809998,53.180393,26.120001,139.199997,73.154999,190.720001,74.220001,171.456665,58.410000,103.790001,112.389999,69.370003,208.899994,71.580002,58.040001,295.450012,55.720001,43.049999,164.910004,31.16,67.279999,412.470001,57.060001,64.510002
2,2015-01-06 00:00:00,44.400002,64.330002,36.130001,39.180000,177.949997,57.450001,255.169998,90.180000,152.899994,124.940002,38.625748,140.449997,334.649994,13.70,58.480000,79.190002,26.125000,63.520000,101.459999,157.119995,94.230003,63.904472,73.620003,50.930000,63.509998,97.650002,71.690002,52.870586,25.750000,139.089996,73.389999,188.309998,71.489998,173.263336,57.070000,103.279999,110.599999,69.720001,208.419998,71.220001,60.320000,291.119995,54.180000,42.660000,164.600006,31.42,66.269997,396.890015,56.650002,64.550003
3,2015-01-07 00:00:00,44.759998,66.930000,37.279999,39.700001,187.929993,59.570000,265.230011,91.980003,158.240005,125.559998,38.810429,143.740005,353.239990,14.03,59.240002,82.029999,26.267500,64.190002,103.839996,162.220001,95.930000,64.291130,74.290001,52.250000,65.000000,99.480003,72.989998,54.333332,26.400000,141.729996,74.870003,194.919998,74.449997,174.213333,57.770000,105.559998,112.540001,69.230003,214.750000,73.000000,61.610001,296.489990,56.110001,42.919998,167.350006,31.85,67.110001,407.720001,57.029999,65.070000
4,2015-01-08 00:00:00,45.680000,67.629997,38.959999,40.889999,183.800003,61.439999,268.959992,92.190002,157.669998,131.550003,39.815319,146.360001,350.250000,14.59,60.720001,84.559998,26.672501,65.599998,109.129997,162.880005,97.970001,65.284309,75.769997,53.540001,66.574997,102.300003,74.930000,55.098038,27.110001,147.830002,75.415001,196.820007,73.209999,177.666672,59.200001,106.389999,115.570000,70.870003,217.199997,74.750000,62.849998,300.720001,56.650002,44.000000,171.169998,32.50,68.879997,403.250000,57.990002,66.779999


In [0]:
stocks.tail()

,Date,ABT,ABBV,ABMD,A,ALXN,ALGN,AGN,ABC,AMGN,ANTM,BAX,BDX,BIIB,BSX,BMY,CAH,CNC,CERN,CI,COO,CVS,DHR,DVA,XRAY,EW,GILD,HCA,HSIC,HOLX,HUM,IDXX,ILMN,INCY,ISRG,IQV,JNJ,LH,LLY,MCK,MDT,MRK,MTD,MYL,PKI,PRGO,PFE,DGX,REGN,RMD,STE
1332,2020-04-20 00:00:00,98.000000,83.989998,166.199997,77.370003,106.720001,196.710007,187.389999,89.620003,236.600006,263.200012,94.139999,263.440002,339.410004,37.250000,61.680000,50.740002,68.419998,70.250000,188.699997,304.700012,62.340000,155.830002,77.519997,41.279999,224.600006,81.260002,110.430000,53.150002,43.240002,367.329987,267.339996,322.880005,101.190002,518.330017,127.510002,151.669998,145.610001,157.789993,140.490005,101.540001,83.099998,707.630005,15.86,84.519997,50.860001,36.080002,95.029999,567.989990,164.279999,157.449997
1333,2020-04-21 00:00:00,94.050003,80.360001,164.500000,73.750000,103.389999,187.259995,184.039993,85.570000,230.750000,249.369995,90.099998,256.500000,329.010010,35.049999,60.590000,48.270000,66.239998,68.589996,177.910004,288.529999,60.250000,153.750000,72.529999,39.660000,216.020004,78.529999,105.470001,52.029999,42.349998,358.859985,259.230011,310.399994,99.459999,496.630005,122.000000,149.679993,148.179993,152.669998,132.440002,97.610001,78.559998,683.429993,15.13,82.639999,49.689999,35.619999,94.459999,544.460022,157.350006,147.779999
1334,2020-04-22 00:00:00,95.480003,81.470001,166.729996,75.169998,103.910004,190.029999,185.339996,86.500000,229.289993,262.959992,90.540001,266.760010,298.010010,35.490002,60.930000,49.040001,68.349998,70.779999,182.589996,294.100006,60.349998,160.250000,72.410004,39.910000,216.520004,81.309998,104.519997,51.709999,42.880001,367.170013,268.119995,311.559998,100.970001,512.130005,125.550003,152.990005,154.130005,156.710007,135.220001,96.080002,79.980003,704.200012,14.91,85.550003,49.619999,36.250000,99.580002,558.080017,160.919998,150.880005
1335,2020-04-23 00:00:00,93.940002,82.040001,170.759995,75.089996,104.139999,188.570007,185.550003,87.260002,232.490006,261.549988,91.320000,265.079987,292.250000,36.400002,60.950001,49.320000,69.239998,71.150002,184.279999,290.269989,61.279999,160.649994,73.349998,38.700001,221.039993,77.779999,104.739998,51.630001,45.380001,366.070007,268.410004,312.709992,101.900002,513.219971,127.669998,155.509995,159.669998,159.929993,136.570007,95.830002,80.879997,703.299988,15.30,87.199997,50.990002,36.689999,100.660004,565.520019,158.729996,149.449997
1336,2020-04-24 00:00:00,94.059998,83.589996,168.520004,75.750000,108.120003,191.970001,186.899994,88.699997,236.279999,264.119995,92.919998,268.170013,297.799988,36.900002,62.250000,50.320000,70.580002,72.750000,188.199997,293.209992,62.310001,165.070007,74.000000,39.310001,223.479996,79.639999,107.830002,52.389999,47.110001,369.089996,271.799988,309.070007,103.480003,514.929993,128.860001,154.860001,163.720001,162.929993,139.940002,99.300003,81.430000,713.380005,15.76,87.550003,51.430000,37.380001,106.620003,566.210022,161.000000,147.820007


In [0]:
#any nulls?
print('There are {} null values\n'.format(stocks.isnull().sum().sum()))

There are 0 null values



In [0]:
#what are the dtypes of the columns
stocks.dtypes

Date     object
ABT     float64
ABBV    float64
ABMD    float64
A       float64
ALXN    float64
ALGN    float64
AGN     float64
ABC     float64
AMGN    float64
ANTM    float64
BAX     float64
BDX     float64
BIIB    float64
BSX     float64
BMY     float64
CAH     float64
CNC     float64
CERN    float64
CI      float64
COO     float64
CVS     float64
DHR     float64
DVA     float64
XRAY    float64
EW      float64
GILD    float64
HCA     float64
HSIC    float64
HOLX    float64
HUM     float64
IDXX    float64
ILMN    float64
INCY    float64
ISRG    float64
IQV     float64
JNJ     float64
LH      float64
LLY     float64
MCK     float64
MDT     float64
MRK     float64
MTD     float64
MYL     float64
PKI     float64
PRGO    float64
PFE     float64
DGX     float64
REGN    float64
RMD     float64
STE     float64
dtype: object

Dates are not in date format. Lets get into date format.

In [0]:
stocks['Date'] = pd.to_datetime(stocks['Date'])
stocks['Date'].dtypes

dtype('<M8[ns]')

In [0]:
print('The shape is {}'.format(stocks.shape))

The shape is (1337, 51)


#### Series to Supervised

In [0]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
  """
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""#n_vars = 1 if type(data) is list else data.shape[1]
  variables = list(data.columns)
  df = data.copy(deep = True)
  cols, names = list(), list()
  # input sequence (t-n, ... t-1)
  for i in range(n_in, 0, -1):
    cols.append(df.shift(i))
    names += ['{}(t-{})'.format(j, i) for j in variables]
  # forecast sequence (t, t+1, ... t+n)
  for i in range(0, n_out):
    cols.append(df.shift(-i))
    if i == 0:
      names += ['{}(t)'.format(j) for j in variables]
    else:
      names += ['{}(t+{})'.format(j, i) for j in variables]
  # put it all together
  agg = pd.concat(cols, axis=1)
  agg.columns = names
  # drop rows with NaN values
  if dropnan:
    agg.dropna(inplace=True)
  return agg

In [0]:
df = series_to_supervised(stocks.loc[:,'ABT':], n_in=5, n_out=2, dropnan=False)

In [0]:
print('new shape is {}'.format(df.shape))
#shape is 350 because it is all 50 stocks of t+1. Remove all except ABT
columns = list(df.columns)[-49:]

new shape is (1337, 350)


In [0]:
df.drop(columns = list(df.columns)[-49:], inplace = True)

In [0]:
df.shape

(1337, 301)